In [ ]:
!pip install rake_nltk
!pip install konlpy
!pip install krwordrank
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from math import log
from nltk.corpus import stopwords
from krwordrank.word import KRWordRank
from krwordrank.word import summarize_with_keywords
from krwordrank.sentence import summarize_with_sentences
from numpy import dot
from numpy.linalg import norm
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt  
from rake_nltk import Rake

In [ ]:
def title_num(code): #원하는 웹툰의 순서를 알아내는 함수
    detail_data = pd.read_csv("webtoon_detail.csv",names =['웹툰제목','점수','작가','장르','추가정보','링크'])
    add_num = list(range(344))
    detail_data["num"] = add_num
    select_data = detail_data[detail_data.index == code]
    num = int(select_data['num'])
    return num

In [ ]:
def prepro_csv(nums): # title_select함수에서 알아낸 코드로 해당 웹툰의 댓글 정보만 읽어내는 함수
    comment_data = pd.read_csv("webtoon_comment.csv",names =['웹툰코드','댓글']) #csv 파일 호출 및 데이터 프레임화
    want_data = comment_data[comment_data['웹툰코드'] == nums] # 선택한 웹툰의 댓글만 추출
    want = want_data['댓글']
    web_array = np.array(want)
    webtoon_df = []
    for i in range(len(web_array)): # 선택한 댓글을 하나의 리스트 변경
        webtoon_df.append(web_array[i])
    return webtoon_df

In [ ]:
def prepro_keyword(texts): # 문장 간소화 함수
    out_list = []
    for token in texts:
        remove_list = []
        remove1 = token.replace("을", "")
        remove2 = remove1.replace("를","")
        remove3 = remove2.replace("[","")
        remove4 = remove3.replace("]","")
        remove5 = remove4.replace("","")
        remove6 = remove5.replace("","")
        out_list.append(remove6)
    return out_list

#위 과정은 밑에서 키워드를 뽑아내기 위해 사용한 summarize_with_sentence가 오로지 문장만 인식합니다. 토크나이저를 사용하면 제대로 인식하지 못합니다. 
#기존의 과정에선 대괄호와 큰따옴표를 제거하는게 힘들어서 토크나이저를 아예 하지 않고 문장 자체에서 특정 단어를 제외하였습니다.

In [ ]:
def keyword_extraction(sentences): # 키워드 추출 함수
    none_list = []
    try:
        penalty = lambda x:0 if (25 <= len(x) <= 80) else 1 # - penalty : 패널티 설정 가능
        stopwords = {'관람객', '너무', '정말', '진짜'} # - stopwords : 키워드에서 제거될 단어, 키워드 벡터를 만들때도 사용하지 않는다.

        keywords, sents = summarize_with_sentences(
            sentences,
            penalty=penalty,
            stopwords = stopwords,
            diversity=0.5,# - diversity : 코싸인 유사도 기준 핵심문장간의 최소 거리, 값이 클수록 다양한 문장 선택
            num_keywords=5, # - num_keywords : 키워드로 추출 될 키워스 갯수를 설정.
            num_keysents=10, # - num_keysents : 핵심문장으로 추출 될 문장의 갯수 설정.
            verbose=False
            )
        keywords_list = []
        for i in keywords:
            keywords_list.append(i)
        return keywords_list
    except:
        return none_list
# 위에서 나온 키워드를 직접 사용하기 위해서 리스트화 하였습니다.


In [ ]:
def tf(term, document): # tf-idf vectorizer 함수
    return document.count(term)

def idf(term):
    df = 0
    for doc in text:
        if term in doc:
            df = df + 1
    return log(len(text)/(df+1))+1

def tfidf(term, document):
    return tf(term, document)*idf(term)

def tfidf_list(text, keywords_list):
    tfidf_list = []
    for doc in text:
        tfidf_list.append([])
        for token in keywords_list:
            tfidf_list[-1].append(tfidf(token, doc))

    tfidf_pd = pd.DataFrame(tfidf_list, columns=keywords_list)# 각 키워드가 문장에 들어있는지 데이터 베이스화
    return tfidf_list, tfidf_pd

In [ ]:
def cos_sim(A, B): #코사인 유사도를 구하는 함수
       return dot(A, B)/(norm(A)*norm(B))

def cos_sim_index(tfidf_list, tfidf_pd, num): #요청한 웹툰과 코사인 유사도가 가장 높은 순서를 찾아내는 함수입니다.
    cos_sim_list = []
    for i in range(len(tfidf_list)):
        if i == num:
            cos_sim_list.append(0)
        else:
            cos_sim_list.append(cos_sim(tfidf_pd.values[num], tfidf_pd.values[i]))
        
    max_num = cos_sim_list.index(max(cos_sim_list))
    i_index = max_num
    print(num,'번과 ',i_index,'번이 가장 유사 모델이다')
    print('코사인 유사도:', cos_sim(tfidf_pd.values[num], tfidf_pd.values[i_index]))
    return i_index

In [ ]:
#돌리는데 10분 넘게 걸리는 코드입니다 처음 시작할때 외에는 돌리지 마세요.
#대신 처음 시작할땐 무조건 돌려야 하는 함수입니다. tfidf 백터의 doc_list에 해당하는 부분입니다.
detail_data = pd.read_csv("webtoon_detail.csv",names =['웹툰제목','점수','작가','장르','추가정보','링크'])
code_list = detail_data.index[1:]# 각 웹툰의 코드 번호만 리스트에 입력합니다
code_nums = [] 
for code in code_list: # 사용하게 좋게 리스트화 시키는 부분입니다.
    code_nums.append(int(code))

vec_list = [] 
for i in code_nums: # 본격적으로 웹툰의 키워드를 하나의 리스트로 합치는 과정입니다.
    text = prepro_csv(i)
    sentence = prepro_keyword(text)
    keywords = keyword_extraction(sentence)
    forvec_list = " ".join(keywords)
    vec_list.append(forvec_list)


In [151]:
def main(webtoon):
    text = prepro_csv(int(webtoon))
    num =(title_num(webtoon) - 1)
    sentence = prepro_keyword(text)
    keywords = keyword_extraction(sentence)
    vector_list, vector_pd = tfidf_list(vec_list, keywords)
    find_num = cos_sim_index(vector_list, vector_pd, num)
    find_web_code = detail_data[detail_data['num'] == find_num + 1].index.tolist()#코사인 유사도가 가장높은 순서를 코드로 재변환 하는 것입니다.
    return find_web_code

In [152]:
webtoon = input()
main(webtoon)

733766
0 번과  136 번이 가장 유사 모델이다
코사인 유사도: 0.8069042084932567


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


['735979']